## Config

In [0]:
import os

base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")
image_classification = os.path.join(base_dir, "ImageClassification")

classes_dict_file = object_detection_dir + "/classes.pkl"
objects_dict_file = object_detection_dir + "/objects.pkl"
images_dict_file = object_detection_dir + "/images.pkl"

## Check to GPU

In [0]:
import tensorflow as tf
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

1.14.0-rc1
Found GPU at: /device:GPU:0


## Mount GDrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from google.colab import auth
auth.authenticate_user()

In [8]:
!gcloud config set project objectdetection-229309

Updated property [core/project].


## Import TF

In [0]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf

In [0]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27708 (delta 20), reused 20 (delta 8), pack-reused 27674
Receiving objects: 100% (27708/27708), 509.12 MiB | 40.09 MiB/s, done.
Resolving deltas: 100% (16965/16965), done.
Checking out files: 100% (2998/2998), done.
W0615 23:16:03.323396 139738524899200 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0615 23:16:03.542270 139738524899200 deprecation_wrapper.py:119] From /content/tf/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.com

In [4]:
#...
sys.path.append('tf/research')
sys.path.append('tf/research/slim')
from object_detection.utils import dataset_util, config_util
from object_detection import model_main

W0616 21:05:07.835777 139928723093376 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0616 21:05:07.855833 139928723093376 deprecation_wrapper.py:119] From tf/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0616 21:05:07.869022 139928723093376 deprecation_wrapper.py:119] From tf/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



## Defines

In [0]:
class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")
  
  #tf records directory
  tf_records_dir = os.path.join(object_detection_dir, "TFRecords")
  #label_map directory
  label_map_dir = os.path.join(object_detection_dir, "LabelMaps")
  
  #class level to be considered
  class_level = [0,1]
  #how often the class of an object needs to occure to be considered
  min_class_occurence = 20
  
  #dataset split
  testing_percentage = 0
  validation_percentage = 10
  
  #pipeline config
  pipeline_config_dir = os.path.join(object_detection_dir, "ConfigFiles")
  
  #model checkpoints
  model_checkpoint_dir = os.path.join(object_detection_dir, "ModelCheckpoints")
  model_dir = os.path.join(object_detection_dir,"Models")
  
  #cloud configs
  cloud_config_dir = os.path.join(object_detection_dir, "CloudConfigFiles")

# Run

In [9]:
from subprocess import call
from object_detection.model_main import FLAGS, main

#!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_mobilenet_v1_075_320x320_holoselecta_sync_colabTPU.config pipeline.config
!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_holoselecta_sync_colabTPU.config pipeline.config
#!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/faster_rcnn_inception_resnet_v2_atrous_holoselecta.config pipeline.config

FLAGS.pipeline_config_path = "pipeline.config"

res = 320
model = "retinanet"

for it in [6, 3, 1]:
  FLAGS.model_dir = "gs://holoselecta_objectdetection/{}_4/{}/{}/".format(model,res,it)
  FLAGS.checkpoint_dir = "gs://holoselecta_objectdetection/{}_4/{}/{}/".format(model,res,it)
  
  ret1 = call('gsutil cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/TFRecords/2019_6_15_15_31_42_test_{}_{}.record gs://holoselecta_objectdetection/holoselecta/dataTPU/val.record'.format(res,it), shell=True)
  ret2 = call('gsutil cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/LabelMaps/2019_6_15_15_31_42.pbtxt gs://holoselecta_objectdetection/holoselecta/dataTPU/label_map.pbtxt', shell=True)
  
  try:
    tf.app.run(main)
  except:
    print("##################  {}  ################".format(it))

W0616 21:06:56.156977 139928723093376 model_lib.py:615] Forced number of epochs for all eval validations to be 1.
I0616 21:06:56.165677 139928723093376 config_util.py:484] Maybe overwriting train_steps: None
I0616 21:06:56.170071 139928723093376 config_util.py:484] Maybe overwriting sample_1_of_n_eval_examples: 1
I0616 21:06:56.173811 139928723093376 config_util.py:484] Maybe overwriting use_bfloat16: False
I0616 21:06:56.176784 139928723093376 config_util.py:484] Maybe overwriting eval_num_epochs: 1
I0616 21:06:56.177992 139928723093376 config_util.py:484] Maybe overwriting load_pretrained: True
I0616 21:06:56.179965 139928723093376 config_util.py:494] Ignoring config override key: load_pretrained
W0616 21:06:56.182318 139928723093376 model_lib.py:631] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
I0616 21:06:56.183302 139928723093376 model_lib.py:666] create_estimator_and_inputs: use_tpu 

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.413
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.323
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.442
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.366
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.413
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.413
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.393
 A

I0616 21:07:35.687544 139928723093376 evaluation.py:275] Finished evaluation at 2019-06-16-21:07:35
I0616 21:07:35.688678 139928723093376 estimator.py:2039] Saving dict for global step 7000: DetectionBoxes_Precision/mAP = 0.33484203, DetectionBoxes_Precision/mAP (large) = -1.0, DetectionBoxes_Precision/mAP (medium) = 0.4424162, DetectionBoxes_Precision/mAP (small) = 0.32328716, DetectionBoxes_Precision/mAP@.50IOU = 0.47706264, DetectionBoxes_Precision/mAP@.75IOU = 0.41273314, DetectionBoxes_Recall/AR@1 = 0.36587152, DetectionBoxes_Recall/AR@10 = 0.41262612, DetectionBoxes_Recall/AR@100 = 0.41262612, DetectionBoxes_Recall/AR@100 (large) = -1.0, DetectionBoxes_Recall/AR@100 (medium) = 0.52019644, DetectionBoxes_Recall/AR@100 (small) = 0.39299223, Loss/classification_loss = 1.1388179, Loss/localization_loss = 0.14601123, Loss/regularization_loss = 0.16057667, Loss/total_loss = 1.4454058, global_step = 7000, learning_rate = 0.009999999, loss = 1.4454058
I0616 21:07:41.282796 13992872309337

##################  6  ################


W0616 21:07:46.660897 139928723093376 model_lib.py:615] Forced number of epochs for all eval validations to be 1.
I0616 21:07:46.668396 139928723093376 config_util.py:484] Maybe overwriting train_steps: None
I0616 21:07:46.671750 139928723093376 config_util.py:484] Maybe overwriting sample_1_of_n_eval_examples: 1
I0616 21:07:46.674842 139928723093376 config_util.py:484] Maybe overwriting use_bfloat16: False
I0616 21:07:46.680249 139928723093376 config_util.py:484] Maybe overwriting eval_num_epochs: 1
I0616 21:07:46.682776 139928723093376 config_util.py:484] Maybe overwriting load_pretrained: True
I0616 21:07:46.683711 139928723093376 config_util.py:494] Ignoring config override key: load_pretrained
W0616 21:07:46.684712 139928723093376 model_lib.py:631] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
I0616 21:07:46.685698 139928723093376 model_lib.py:666] create_estimator_and_inputs: use_tpu 

...

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.61s).
Accumulating evaluation results...


I0616 21:09:49.263280 139928723093376 evaluation.py:275] Finished evaluation at 2019-06-16-21:09:49
I0616 21:09:49.264422 139928723093376 estimator.py:2039] Saving dict for global step 7000: DetectionBoxes_Precision/mAP = 0.16227232, DetectionBoxes_Precision/mAP (large) = -1.0, DetectionBoxes_Precision/mAP (medium) = 0.24251124, DetectionBoxes_Precision/mAP (small) = 0.13900942, DetectionBoxes_Precision/mAP@.50IOU = 0.26093778, DetectionBoxes_Precision/mAP@.75IOU = 0.17436257, DetectionBoxes_Recall/AR@1 = 0.20167004, DetectionBoxes_Recall/AR@10 = 0.259546, DetectionBoxes_Recall/AR@100 = 0.26031408, DetectionBoxes_Recall/AR@100 (large) = -1.0, DetectionBoxes_Recall/AR@100 (medium) = 0.33497533, DetectionBoxes_Recall/AR@100 (small) = 0.22482629, Loss/classification_loss = 1.2889088, Loss/localization_loss = 0.26891738, Loss/regularization_loss = 0.15119042, Loss/total_loss = 1.7090168, global_step = 7000, learning_rate = 0.009999999, loss = 1.7090168


DONE (t=0.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.261
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.202
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.335
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


I0616 21:09:54.877320 139928723093376 estimator.py:2099] Saving 'checkpoint_path' summary for global step 7000: gs://holoselecta_objectdetection/retinanet_4/320/1/model.ckpt-7000
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

I0616 21:09:55.723190 139928723093376 model_lib.py:794] Evaluation finished after training step 7000


##################  1  ################


In [0]:
from object_detection.model_main import FLAGS, main

!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_mobilenet_v1_075_320x320_coco14_sync_colabTPU.config pipeline.config

FLAGS.pipeline_config_path = "pipeline.config"
FLAGS.model_dir = "gs://holoselecta_objectdetection/mobilenetv2/320/100/"
FLAGS.checkpoint_dir = "gs://holoselecta_objectdetection/mobilenetv2/320/100/"

tf.app.run(main)

INFO:tensorflow:Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu False
INFO:tensorflow:Using config: {'_model_dir': 'gs://holoselecta_objectdetection/mobilenetv2/320/100/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




INFO:tensorflow:Evaluation finished after training step 7000


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Copy & Save

In [0]:
import shutil
import os

timestamp = "2019_5_7_19_42_14"

with open("test_model/{}/README.md".format(timestamp), "+w") as f:
  f.write("# Model Description  \n")
  f.write("Model: SSD, Resnet 50, Focal Point Network (FPN), Shared Box Predictor, (aka RetinaNet)  \n")
  f.write("Input Resolution: 640 x 640  \n")
  f.write("Classes: 38  \n")
  f.write("Training Images: _Automat  \n")
  f.write("Dataset Timestamp: 2019_5_7_12_50_41  \n")
  f.write("Training Steps: 7000  \n")
  f.write("Final mAP@.50IOU = 0.69 ; mAP@.75IOU = 0.60  \n")
  f.write("Max Score Steps: 7000")

shutil.copy("model/frozen_inference_graph.pb", "test_model/{}/frozen_inference_graph.pb".format(timestamp))
shutil.copy("data/label_map.pbtxt", "test_model/{}/label_map.pbtxt".format(timestamp))
shutil.copy("pipeline.config", "test_model/{}/pipeline.config".format(timestamp))
shutil.copy("timestamp", "test_model/{}/timestamp".format(timestamp))

shutil.copytree("test_model/{}".format(timestamp), os.path.join(FLAGS_OD.model_dir, "Good", timestamp))

'/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Models/Good/2019_5_7_19_42_14'

# Stuff

These resolutions are problematic because we keep an entire queue of images in memory, not just the batch that you are currently training on. See e.g. the queue_capacity and min_after_dequeue parameters in https://github.com/tensorflow/models/blob/master/object_detection/protos/input_reader.proto